In [3]:
!pip install torchtext==0.4.0

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support
     |████████████████████████████████| 51kB 15.9MB/s eta 0:00:01
  Created wheel for torchtext: filename=torchtext-0.4.0-cp27-none-any.whl size=52131 sha256=4ea0aecfa9bf43f28bf88597903bc5e7459546a16855b5f4cd575ec214401144
  Stored in directory: /home/ec2-user/.cache/pip/wheels/7f/0b/a7/53f554f01d205ac7039ef96028eb886f52e235cdfae5ecf7ef
Successfully built torchtext
You should consider upgrading via the 'pip install --upgrade pip' command.


In [5]:
import torchtext
import numpy as np

# 0.1 라이브러리 임포트

In [7]:
import torch
import torch.nn as nn
import torchvision.datasets as dset
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torch.autograd import Variable
import numpy as np

In [33]:
from data_loader import DataLoader

# 0.2 하이퍼파라미터 셋팅

In [34]:
# hyper-parameters
batch_size = 128
num_epochs = 10

word_vec_size = 256
dropout_p = 0.3

hidden_size = 512
num_layers = 4

## yhk 추가
learning_rate = 0.001 # default is 0.001

In [35]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 1. SMS train, test dataset 가져오기

In [36]:
from data_loader import DataLoader

In [37]:
loaders = DataLoader(
    train_fn = './sms.maxlen.uniq.shuf.train.tsv',
    batch_size = batch_size,
    valid_ratio = .2,
    device = -1,
    max_vocab = 999999,
    min_freq = 5,)

In [38]:
test_loaders = DataLoader(
    train_fn = './sms.maxlen.uniq.shuf.test.tsv',
    batch_size = batch_size,
    valid_ratio = .01, # 모두 train
    device = -1,
    max_vocab = 999999,
    min_freq = 5,)

# 2. 대략적인 데이터 형태

In [39]:
print("|train| =", len(loaders.train_loader.dataset),
       '|valid| =', len(loaders.valid_loader.dataset))

vocab_size = len(loaders.text.vocab)
num_classes = len(loaders.label.vocab)
print('|vocab| =', vocab_size, '|classes| =', num_classes)

('|train| =', 3722, '|valid| =', 930)
('|vocab| =', 1547, '|classes| =', 2)


# 3. 데이터 로드 함수
학습시킬 때 batch_size 단위로 끊어서 로드하기 위함

## 데이터 로드함수 이해하기

In [57]:
n = 3 # 샘플로 출력할 데이터 개수
for i, data in enumerate(loaders.train_loader):
    labels = data.label
    texts = data.text
    
    if i > n:
        break
    print("[%d]"%i)
    print("한 번에 로드되는 데이터 크기")
    print(len(labels))
    
    # print
    for j in range(n):
        label = labels[j].numpy() # tensor -> numpy 로 변환
        text = texts[j].numpy()
        print("label: ")
        print(label)
        print("text: ")
        print(text.shape)

[0]
한 번에 로드되는 데이터 크기
128
label: 
0
text: 
(7,)
label: 
0
text: 
(7,)
label: 
0
text: 
(7,)
[1]
한 번에 로드되는 데이터 크기
128
label: 
0
text: 
(19,)
label: 
0
text: 
(19,)
label: 
0
text: 
(19,)
[2]
한 번에 로드되는 데이터 크기
128
label: 
0
text: 
(16,)
label: 
0
text: 
(16,)
label: 
0
text: 
(16,)
[3]
한 번에 로드되는 데이터 크기
128
label: 
0
text: 
(34,)
label: 
0
text: 
(34,)
label: 
1
text: 
(34,)


# 4. 모델 선언

In [52]:
# Recurrent neural network(many-to-one)
class RNN(nn.Module):
    def __init__(self,
                input_size,
                word_vec_size,
                hidden_size,
                n_classes,
                num_layers=4,
                dropout_p=0.3):
        super(RNN, self).__init__()
        
        self.input_size = input_size
        self.word_vec_size = word_vec_size
        self.hidden_size = hidden_size
        self.n_classes = n_classes
        self.num_layers = num_layers
        self.dropout_p = dropout_p
        
        #입력 차원, 출력 차원
        self.emb = nn.Embedding(input_size, word_vec_size)
        self.lstm = nn.LSTM(input_size = word_vec_size,
                            hidden_size = hidden_size,
                            num_layers = num_layers,
                            dropout = dropout_p,
                            batch_first = True,
                            bidirectional = True)
        self.fc = nn.Linear(hidden_size*2, num_classes)
        self.activation = nn.LogSoftmax(dim=-1)
        
        # 시험문제 빈칸 나올수도..?
    def forward(self, x):
        x = self.emb(x)    
        x,_ = self.lstm(x)
        out = self.activation(self.fc(x[:,-1]))
            
        return out

In [53]:
model = RNN(input_size=vocab_size,
           word_vec_size=word_vec_size,
           hidden_size = hidden_size,
           n_classes = num_classes,
           num_layers = num_layers,
           dropout_p = dropout_p)

In [54]:
def ComputeAccr(dloader, imodel):
    correct = 0
    total = 0
    model.eval() # test mode
    for i, data in enumerate(dloader):
        texts = data.text.to(device)
        labels = data.label.to(device)
        
        #Forward prop
        output = model(texts)
        _, output_index = torch.max(output, 1)
        
        total += labels.size(0)
        correct += (output_index == labels).sum().float()
        
        model.train()
        return (100*correct/total).numpy() # tensor -> numpu

In [56]:
print("Accuracy of Test Data : %.2f" %ComputeAccr(loaders.valid_loader, model))

Accuracy of Test Data : 10.16


# 5. loss, optimizer


In [58]:
loss_func = nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

# 6. 학습

In [59]:
# Train the model
total_step = len(loaders.train_loader)
for epoch in range(num_epochs):
    for i, data in enumerate(loaders.train_loader):
        texts = data.text.to(device)
        labels = data.label.to(device)
        
        print("[%d]" %i)
        
        outputs = model(texts)
        loss = loss_func(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if(i+1)  % 10 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {: .4f}, Accr : {:.2f}'.
                 format(epoch+1, num_epochs, i+1, total_step, loss.item(), ComputeAccr(loaders.valid_loader, model)))

[0]
[1]
[2]
[3]
[4]
[5]
[6]
[7]
[8]
[9]
Epoch [1/10], Step [10/30], Loss:  0.3428, Accr : 96.88
[10]
[11]
[12]
[13]
[14]
[15]
[16]
[17]
[18]
[19]
Epoch [1/10], Step [20/30], Loss:  0.3182, Accr : 96.88
[20]
[21]
[22]
[23]
[24]
[25]
[26]
[27]
[28]
[29]
Epoch [1/10], Step [30/30], Loss:  0.2008, Accr : 96.88
[0]
[1]
[2]
[3]
[4]
[5]
[6]
[7]
[8]
[9]
Epoch [2/10], Step [10/30], Loss:  0.0522, Accr : 96.88
[10]
[11]
[12]
[13]
[14]
[15]
[16]
[17]
[18]
[19]
Epoch [2/10], Step [20/30], Loss:  0.9393, Accr : 96.88
[20]
[21]
[22]
[23]
[24]
[25]
[26]
[27]
[28]
[29]
Epoch [2/10], Step [30/30], Loss:  0.4107, Accr : 96.88
[0]
[1]
[2]
[3]
[4]
[5]
[6]
[7]
[8]
[9]
Epoch [3/10], Step [10/30], Loss:  0.1270, Accr : 96.88
[10]
[11]
[12]
[13]
[14]
[15]
[16]
[17]
[18]
[19]
Epoch [3/10], Step [20/30], Loss:  0.6070, Accr : 78.12
[20]
[21]
[22]
[23]
[24]
[25]
[26]
[27]
[28]
[29]
Epoch [3/10], Step [30/30], Loss:  0.4024, Accr : 96.88
[0]
[1]
[2]
[3]
[4]
[5]
[6]
[7]
[8]
[9]
Epoch [4/10], Step [10/30], Loss:  0

# 7. 테스트

In [60]:
print("Accuracy of Test Data : %.2f" %ComputeAccr(loaders.valid_loader, model))

Accuracy of Test Data : 96.88


# 8. 학습된 파라미터 저장

In [61]:
netname = './nets/rnn_weight.pkl'
torch.save(model, netname, )

/home/ec2-user/anaconda3/envs/pytorch_p27/lib/python2.7/site-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


# 9. 학습된 파라미터 로드
실무에서 학습된 파라미터 로드하고 싶다면 : 5,6,8 과정 생략한 채 실행
    

In [ ]:
netname = './nets/rnn_weight.pkl'
model = torch.load(netname)
print("Accuracy of Test Data : %.2f" %ComputeAccr(loaders.valid_loader, model))